We use an Online learner and see if it helps. To train Vowpal Wabbit on all data, we use Pandas' handy chunk functionality to load in the data.


In [ ]:
!pip uninstall vowpalwabbit -y
!pip install vowpalwabbit --no-index --find-links=file:../input/vowpalwabbit/vowpalwabbit-9.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

In [ ]:
!pip install vowpalwabbit --no-index --find-links=file:../input/vowpalwabbit/vowpalwabbit-9.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

The imports fail at first, but the try-except statement gets around that.

In [ ]:
try:
    from vowpalwabbit.dftovw import DFtoVW
    from vowpalwabbit import Workspace
    import vowpalwabbit
    import pandas as pd
    import numpy as np
    import gc
except Exception:
    from vowpalwabbit.dftovw import DFtoVW
    from vowpalwabbit import Workspace
    import vowpalwabbit
    import pandas as pd
    import numpy as np
    import gc

In [ ]:
target = "target"

This part comes from the Vowpal Wabbit documentation.

In [ ]:
model = vowpalwabbit.Workspace(quiet=True)

The training loo....

In [ ]:
done = 0
chunksize = 1000
with pd.read_csv("../input/ubiquant-market-prediction/train.csv", chunksize=chunksize) as reader:
    for chunk in reader:
        print(f"Start batch beginning from {done+1}")
        del chunk["row_id"]
        del chunk["time_id"]
        chunk["investment_id"] = chunk["investment_id"].astype(str) # should be interpreted as category
        chunk[chunk == "nan"] = np.nan
        
        x = chunk.columns.to_list()
        y = target
        x.remove(target)
        converter = DFtoVW.from_colnames(df=chunk, y=y, x=x)
        examples = converter.convert_df()
        #print(examples)
        for ex in examples:
            model.learn(ex)
        
        done += chunksize

The inference loop.

In [ ]:
import ubiquant
env = ubiquant.make_env()
# initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

Here we encountered an issue. We did not find a way to use the DFtoVW function without providing a dummy target.

Performance in geeral has been really bad with Vowpal Wabbit. Probably not worth investigating further.

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    del test_df["row_id"]
    test_df["investment_id"] = test_df["investment_id"].astype(str) # should be interpreted as category
    test_df[test_df == "nan"] = np.nan
    test_df[target] = 0
    converter = DFtoVW.from_colnames(df=test_df, x=x, y=y)
    examples = converter.convert_df()
    for ex in examples:
            pred = model.predict(ex)
            print(pred)
    sample_prediction_df['target'] = pred # attach predictions
    env.predict(sample_prediction_df)   # register your predictions
    del test_df
    del sample_prediction_df
    _ = gc.collect()